In [1]:
from maliboo import BayesianOptimization, UtilityFunction
import numpy as np
import os

import matplotlib.pyplot as plt
from matplotlib import gridspec
%matplotlib inline

In [2]:
def target(x,y):
    #return (np.exp(-(x - 2)**2) + np.exp(-(x - 6)**2/10) + 1/ (x**2 + 1))
    #return  -x**2
    #return -np.abs(x)
    #return np.cos(x)
    #return -x**2 -y**2
    return -(x + y)

# x = np.linspace(-7, 7, 10000).reshape(-1, 1)
# y = target(x)

# plt.plot(x,y)

In [3]:
import warnings
def posterior(optimizer, x_obs, y_obs, grid):
    optimizer._gp.fit(x_obs, y_obs)
    mu, sigma = optimizer._gp.predict(grid, return_std=True)
    return mu, sigma

def plot_gp(optimizer, x, y, show_minimized=False, style='transparent'):

    if style == 'classic':
        plt.style.use(style)
    elif style == 'dark':
        plt.style.use('dark_background')
    elif style == 'transparent':
        pass
    else:
        raise ValueError("Wrong style")

    # Swap sign of stuff
    sign = -1 if show_minimized else 1
    
    # Set up complete plot
    plt.rcParams.update({'font.size': 20})

    fig = plt.figure(figsize=(16, 10))
    steps = len(optimizer.space)
    gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1]) 
    axis = plt.subplot(gs[0])
    acq = plt.subplot(gs[1])

    ## MAIN PLOT
    # Plot true objective
    target_col = 'limegreen' if style == 'dark' else 'gray'
    axis.plot(x, sign*y, linewidth=3.5, label='objective function',
              color=target_col)
    # Plot observations
    x_obs = np.array([[res["x"]] for res in optimizer.res])
    y_obs = np.array([res["target"] for res in optimizer.res])
    axis.scatter(x_obs, sign*y_obs, marker='o', s=300,
                 label='observations', color='red', zorder=10)   
    # Plot mean function
    mu, sigma = posterior(optimizer, x_obs, y_obs, x)
    mu = sign * mu
    axis.plot(x, mu, dashes=(12, 3.5), color='blue', lw=2.5, label='mean prediction')
    # Plot credible interval
    quant = 1.96
    axis.fill(np.concatenate([x, x[::-1]]), 
              np.concatenate([mu - quant * sigma, (mu + quant * sigma)[::-1]]),
        alpha=.6, fc='paleturquoise', ec='None', label='95% credible interval')
    # Set parameters
    xlim = (-2,10)
    axis.set_xlim(xlim)
    axis.set_xlabel('x')
    axis.set_ylim(-1.61, 0.01)
    axis.set_ylabel('objective function')
    axis.legend(loc=2, bbox_to_anchor=(1.01, 1), borderaxespad=0.)

    ## ACQUISITION PLOT
    utility_function = UtilityFunction(kind="ucb", kappa=5, xi=0)
    utility = utility_function.utility(x, optimizer._gp, 0)
    x_next = x[np.argmax(utility)]
    print(x_next)
    acq.plot(x, utility, label='acquisition function', lw=2, color='fuchsia')
    acq.scatter(x_next, np.max(utility), marker='x', s=300, 
             label='next observation', ec='red', lw=4, zorder=10)
    acq.set_xlim(xlim)
    acq.set_ylim((0, 2.501))
    acq.set_ylabel('utility')
    acq.set_xlabel('x')
    acq.legend(loc=2, bbox_to_anchor=(1.01, 1))

    # Vertical lines in both
    axis.vlines(x_next, sign*0, sign*1.6, lw=1.5, linestyle=(0, (10, 3.5)), color='silver')
    acq.vlines(x_next,       0,        5, lw=1.5, linestyle=(0, (10, 3.5)), color='silver')

    # Overlay vertical Gaussian distribution
    ypos_idx = np.abs(x-x_next).argmin()
    ypos = mu[ypos_idx]
    var = 0.6 * sigma[ypos_idx]  # I'm cheating a little bit with the variance
                                 # for representation purposes
    bound = 0.5
    supp = np.linspace(-bound, bound, 1000).reshape(-1, 1)
    gaussian = np.exp((-supp ** 2.0) / (2 * var ** 2.0))
    gaussian /= gaussian.max()
    gaussian *= -0.5  # height
    axis.plot(gaussian + x_next, supp + ypos, lw=1.5, color='silver')

    # Save to file
    for fmt in ('png', 'svg'):
        filename = f'bo_{style}_{len(x_obs)}.{fmt}'
        plt.savefig(os.path.join('resources', filename), bbox_inches='tight', dpi=300)

# # UCB
# # Initialize optimizer
# optimizer = BayesianOptimization(target, {'x': (-2, 10)},
#                                  random_state=272242)
# # First rounds of optimization
# optimizer.maximize(init_points=3, n_iter=1, kappa=5)
# for style in ('classic', 'dark'):  # ('transparent', 'classic', 'dark')
#     plot_gp(optimizer, x, y, show_minimized=True, style=style)

# EB
# Initialize optimizer again
optimizer = BayesianOptimization(target, {'x': (-5,5)
                                          , 'y': (-5,5)
                                           }, 
                                 random_state=27222,barrier_func={'constraint1': lambda x,y: 3/2 -x-2*y - 1/2*np.sin(2*np.pi*(x**2 - 2*y)),
                                                                  'constraint2': lambda x,y: x**2 + y**2 - 3/2
                                                                  }, debug = True)
# Other rounds of optimization
with warnings.catch_warnings():
    warnings.simplefilter("ignore", RuntimeWarning)
    warnings.simplefilter("ignore", FutureWarning)
    optimizer.maximize(init_points=1000, n_iter=40,acq='eb',acq_info={'static_lambda':False})


Initializing TargetSpace with bounds: {'x': (-5, 5), 'y': (-5, 5)}
initialize_dataset(): dataset is None
TargetSpace initialization completed
BayesianOptimization initialization completed
Starting maximize()
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
_prime_queue(): initializing 1000 random points
Uniform randomly sampled point: value [[ 2.22179014 -3.42876464]]
Uniform randomly sampled point: value [[-2.3900672  -4.19462688]]
Uniform randomly sampled point: value [[ 0.4791047  -3.26929793]]
Uniform randomly sampled point: value [[1.16139873 4.71492313]]
Uniform randomly sampled point: value [[-3.55204527 -1.23007182]]
Uniform randomly sampled point: value [[-0.84787582 -2.73280891]]
Uniform randomly sampled point: value [[ 1.28035784 -3.04821078]]
Uniform randomly sampled point: value [[1.07550063 3.90074381]]
Uniform randomly sampled point: value [[-4.98123446 -0.63829644]]
Uniform randomly sampled point: value [[-1.23800653 -0

In [ ]:
optimizer.max

{'target': -0.8484848484848486,
 'params': {'x': 0.4242424242424243, 'y': 0.4242424242424243}}

In [ ]:
gp_barrier = optimizer._gps_barriers


In [ ]:
optimizer.space.x_grid

array([[-2.        , -1.95959596],
       [-1.91919192, -1.87878788],
       [-1.83838384, -1.7979798 ],
       ...,
       [ 2.        ,  2.        ],
       [ 2.        ,  2.        ],
       [ 2.        ,  2.        ]])

In [ ]:
gp_function = optimizer._gp

In [ ]:
def ac(x,gp_barrier,gp_function):
    mu_c,stdc = gp_barrier.predict(np.array([x]), return_std= True)
    mu_f,stdf = gp_function.predict(np.array([x]), return_std= True)
    return -mu_f - (stdf**2)*(np.log(-mu_c) - stdc**2/(2*mu_c**2))

In [ ]:
import scipy

In [ ]:
x_draw = np.linspace(-4,4
                     , 1000)
ac_eval = []
for i in range(len(x_draw)):
    ac_eval.append(ac([x_draw[i]],gp_barrier,gp_function))


AttributeError: 'list' object has no attribute 'predict'

In [ ]:
x = np.linspace(-10,10,10)
y = np.linspace(-10,10,10)
z = np.linspace(-10,10,10)
a = np.meshgrid(*[x,y,z])
a = np.array(a).reshape(3,-1)




In [ ]:
a

array([[-10.        , -10.        , -10.        , ...,  10.        ,
         10.        ,  10.        ],
       [-10.        , -10.        , -10.        , ...,  10.        ,
         10.        ,  10.        ],
       [-10.        ,  -7.77777778,  -5.55555556, ...,   5.55555556,
          7.77777778,  10.        ]])

In [ ]:
len(a)

3

In [ ]:
plt.plot(x_draw,np.array(ac_eval))

In [ ]:
x_grid = np.array([np.linspace(bound[0],bound[1], 100) for bound in bounds]).T
gp_barrier_evaluations = np.array([gp.predict(x_grid) for gp in gp_barrier ])

In [ ]:
col_mask = np.logical_and((gp_barrier_evaluations <= 0)[0,:],(gp_barrier_evaluations <= 0)[1,:] )
gp_barrier_evaluations[:,col_mask]

In [ ]:
gp_barrier_evaluations[:,col_mask].T

In [ ]:
optimizer.space.bounds